In [1]:
from pathlib import Path
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization ,hashes
from cryptography.hazmat.primitives.asymmetric import padding
from loguru import logger
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

import os


from zip_utils import *

In [2]:
public_key_path = Path(".") / "keys"/ "public_key.pem"
private_key_path = Path(".") / "keys"/ "private_key.pem"

public_key_path.parent.mkdir(parents=True, exist_ok=True)
private_key_path.parent.mkdir(parents=True, exist_ok=True)

# Generate private key
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,  # or 4096 for higher security
)

# Derive public key from private key
public_key = private_key.public_key()

# Step 3.1: Save the private key to a file
with open(private_key_path, "wb") as f:
    f.write(
        private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.NoEncryption(),
        )
    )
logger.info(f"Private key saved to {private_key_path}")

# Step 3.2: Save the public key to a file
with open(public_key_path, "wb") as f:
    f.write(
        public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo,
        )
    )
logger.info(f"Public key saved to {public_key_path}")

2025-05-19 11:02:55.179 | INFO     | __main__:<module>:25 - Private key saved to keys/private_key.pem
2025-05-19 11:02:55.180 | INFO     | __main__:<module>:35 - Public key saved to keys/public_key.pem


In [3]:
folder_path = Path(".") / "Organic Crop Data"

In [4]:
zip_bytes = zip_to_bytes(files_or_dirs=folder_path, base_dir="Organic Crop Data")

In [5]:
get_files_from_zip(zip_bytes)

{'config.yaml': b'code:\n  entrypoint: main.py\n\ndata:\n  - [do1@openmined.org, 5a0c7c32-f9b6-4d74-bdbf-42547004c2b4]\n  - [do2@openmined.org, d051a30e-8e84-4240-8f45-c5d918439d0f]\n\noutput:\n  - do1@openmined.org\n  - do2@openmined.org\n',
 'code/entrypoint.py': b''}

In [6]:
def encrypt_bytes(zip_bytes):
    # Step 1: Generate a random AES key
    aes_key = AESGCM.generate_key(bit_length=256)
    
    # Step 2: Encrypt the zip data with AES-GCM
    aesgcm = AESGCM(aes_key)
    nonce = os.urandom(12)  # GCM standard nonce size
    ciphertext = aesgcm.encrypt(nonce, zip_bytes, associated_data=None)
    
    # Step 3: Encrypt the AES key with RSA public key
    encrypted_key = public_key.encrypt(
        aes_key,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None,
        ),
    )
    
    # Step 4: Save (or send) nonce + encrypted AES key + ciphertext
    with open("organic.enc", "wb") as f:
        f.write(len(encrypted_key).to_bytes(2, "big"))  # length of key prefix
        f.write(encrypted_key)
        f.write(nonce)
        f.write(ciphertext)

In [7]:
encrypt_bytes(zip_bytes)

In [8]:
def decrypt_bytes():
    # Step 1: Read the encrypted file
    with open("organic.enc", "rb") as f:
        key_len = int.from_bytes(f.read(2), "big")
        encrypted_key = f.read(key_len)
        nonce = f.read(12)
        ciphertext = f.read()
    
    # Step 2: Decrypt the AES key using private key
    aes_key = private_key.decrypt(
        encrypted_key,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None,
        ),
    )
    
    # Step 3: Decrypt the ZIP file
    aesgcm = AESGCM(aes_key)
    zip_bytes = aesgcm.decrypt(nonce, ciphertext, associated_data=None)

    return zip_bytes

In [9]:
dec_bytes = decrypt_bytes()

In [10]:
extract_folder = Path(".") / "Temp"
extract_zip(zip_data=dec_bytes, target_dir=extract_folder)